In [43]:
import sqlite3
import pandas as pd
import numpy as np
import plotly.express as px
import geopandas as gpd
import plotly.io as pio
import plotly.graph_objects as go

In [2]:
conn = sqlite3.connect('earthquake.db')
conn

In [3]:
# Tables for wells
wells = pd.read_csv('../data/wells.csv')
wells.to_sql('wells', conn, if_exists='replace', index=False)

well_locations = pd.read_csv('../data/well_locations.csv')
well_locations.to_sql('well_locations', conn, if_exists='replace', index=False)

well_intake = pd.read_csv('../data/well_intake.csv')
well_intake.to_sql('well_intake', conn, if_exists='replace', index=False)

operators = pd.read_csv('../data/operators.csv')
operators.to_sql('operators', conn, if_exists='replace', index=False)

orders = pd.read_csv('../data/orders.csv')
orders.to_sql('orders', conn, if_exists='replace', index=False)

9685

In [4]:
# Tables for earthquakes

events_df1 = pd.read_csv('../data/events1.csv')
events_df2 = pd.read_csv('../data/events2.csv')
combined_events = pd.concat([events_df1, events_df2], ignore_index=True)
combined_events.to_sql('events', conn, if_exists='replace', index=False)

event_locations1_df = pd.read_csv('../data/event_locations1.csv')
event_locations2_df = pd.read_csv('../data/event_locations2.csv')
combined_event_locations = pd.concat([event_locations1_df, event_locations2_df], ignore_index=True)
combined_event_locations.to_sql('event_locations', conn, if_exists='replace', index=False)

magnitude1_df = pd.read_csv('../data/magnitude1.csv')
magnitude2_df = pd.read_csv('../data/magnitude2.csv')
combined_magnitude = pd.concat([magnitude1_df, magnitude2_df], ignore_index=True)
combined_magnitude.to_sql('magnitude', conn, if_exists='replace', index=False)

15580

In [5]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type= 'table';", conn)

,name
0,wells
1,well_locations
2,well_intake
3,operators
4,orders
5,events
6,event_locations
7,magnitude


In [11]:
# Oklahoma earthquake locations
query1 = """SELECT
    el.event_id, el.latitude, el.longitude
FROM event_locations el
JOIN events e ON e.event_id = el.event_id
WHERE e.year BETWEEN 2010 and 2017
AND state = 'Oklahoma'
"""

result1 = pd.read_sql(query1, conn)
result1 

,event_id,latitude,longitude
0,usp000h5kf,35.6350,-97.1850
1,usp000h5y9,35.5390,-97.2590
2,usp000h63j,35.5390,-97.2380
3,usp000h65q,35.5760,-97.2500
4,usp000h65r,35.5550,-97.2490
...,...,...,...
7599,us10006qzl,36.9246,-97.8946
7600,us10006qzw,36.3735,-96.8187
7601,us10006r04,36.4122,-96.8824
7602,us10006r59,36.9393,-97.8960


In [12]:
# Oklahoma injection well locations
query2 = """SELECT DISTINCT
    well_locations.well_number, well_locations.latitude, well_locations.longitude
FROM well_locations
JOIN orders ON orders.well_number = well_locations.well_number
WHERE orders.year BETWEEN 2010 AND 2017
"""
result2 = pd.read_sql(query2, conn)
result2

,well_number,latitude,longitude
0,A-1,33.906562,-97.368262
1,A-1,34.062032,-97.362772
2,A-1,34.208403,-97.368837
3,A-1,34.208860,-97.491591
4,A-1,34.292751,-97.710838
...,...,...,...
7372,1-9-SWD,36.753678,-98.822380
7373,#2-16,36.738797,-98.305810
7374,21-Feb,36.711033,-98.713170
7375,29-B,34.181744,-97.332454


In [22]:
fig1 = px.scatter_map(result1,
            lat=result1.latitude,
            lon=result1.longitude,
            hover_name = 'event_id',
            color_discrete_sequence=['red'])
fig1.update_layout(showlegend=True)
fig1.show()

In [23]:
fig2 = px.scatter_map(result2,
            lat=result2.latitude,
            lon=result2.longitude,
            hover_name = 'well_number',
            color_discrete_sequence=['blue'])
fig2.update_layout(showlegend=True)

fig2.show()

In [ ]:
fig1.add_trace(fig2.data[0])
fig1.data[0].name = 'Oklahoma earthquakes'
fig1.data[1].name = 'Oklahoma injection wells'
fig1.update_layout(title_text='Overlay of Oklahoma Earthquakes and Injection Wells ',
    showlegend=True)

fig1.show()



In [45]:
# Number of orders of injection wells approved from 2000 to 2017
query3 = """SELECT
   year, COUNT(approval_date)
FROM orders o
WHERE o.year > 2009
GROUP BY o.year
ORDER BY o.year
"""

result3 = pd.read_sql(query3, conn)
result3 

,year,COUNT(approval_date)
0,2010,293
1,2011,437
2,2012,447
3,2013,403
4,2014,392
5,2015,246
6,2016,217
7,2017,194


In [ ]:
# OK earthquake events (magnitude and depth) by year

query4 = """SELECT
   e.year, m.magnitude, m.depth
FROM events e
JOIN magnitude m ON m.event_id = e.event_id
JOIN event_locations el ON el.event_id = m.event_id   
WHERE state = "Oklahoma" and year > 2009

"""

result4 = pd.read_sql(query4, conn)
result4 

,year,magnitude,depth
0,2010.0,2.8,5.00
1,2010.0,2.8,5.00
2,2010.0,3.3,5.00
3,2010.0,3.8,8.00
4,2010.0,3.7,8.00
...,...,...,...
7599,2016.0,3.9,2.62
7600,2016.0,2.3,4.69
7601,2016.0,1.4,4.41
7602,2016.0,3.0,2.32
